In [1]:
#pulls verses from TaNaKh using the Sefaria API

In [1]:
import json, urllib.request

In [2]:
aleph_bet = ['א','ב','ג','ד','ה','ו','ז','ח','ט','י','כ','ך','ל','מ','ם',
             'נ','ן','ס','ע','פ','ף','צ','ק','ר','ש','ת','װ','ױ','ײ','יִ',
             'ﬡ','ﬢ','ﬣ','ﬤ','ﬥ','ﬦ','ﬧ','ﬨ','שׁ','שׂ','שּׁ','שּׂ','אַ','אָ',
             'גּ','דּ','הּ','וּ','זּ','טּ','יּ','ךּ','כּ','לּ','מּ','נּ','סּ','ףּ',
             'פּ','צּ','שּ','תּ','וֹ','בֿ','כֿ','פֿ','ﭏ','בּ', 'קּ']

cant = ['֑','֒','֓','֔','֕','֖','֗','֘','֙','֚','֛','֜',
             '֝','֞','֠','֡','֢','֣','֤','֥','֦','֧','֨','֩','֪','֫','֬','֭','֯','׃']

cant2 = cant[:28]

vowel = ['ְ','ֱ','ֲ','ֳ','ִ','ֵ','ֶ','ַ','ָ','ֹ','ֺ','ֻ','ּ','ֽ','־','ֿ','ׁ','ׂ','ׄ','ׅ','ׇ']

letter_with_dagesh = ['שּׁ','שּׂ','גּ','דּ','זּ','טּ','יּ','כּ','לּ','מּ','נּ','סּ','פּ','צּ','שּ','תּ','בּ','קּ']
rafe = 'ֿ'
shva = vowel[0]
chataf_vowels = vowel[1:4]
short_vowels = [vowel[4],vowel[6],vowel[7],vowel[11],vowel[20]]
long_vowels = ['וֹ','וּ', vowel[5], vowel[8], aleph_bet[9], 'ֹ']
vowels_limited = vowel[0:4]+short_vowels+long_vowels
dagesh = 'ּ'
maqqaf = '־'
meteg = 'ֽ'
dot = '֯'
silluq = '׃'
colon = ':'
other_char = ['ׂ','ׁ','ֿ','֯','ׅ','']
all_char = aleph_bet+cant+vowel+other_char

special_characters = ['{', '}','\\']

gutterals = ['א','ה','ח','ע','ר' ,'ﬡ','ﬣ','ﬧ']    

In [3]:
letters_w_final = ['כ','מ','נ','פ','צ']
final_letters = ['ך','ם','ן','ף','ץ']

In [4]:
shem = 'יהוה'
shem_vowels = 'יְהֹוָה'
yy_vowels = aleph_bet[9]+vowel[0]+aleph_bet[9]+vowel[8]

kal_backwards_dagesh = 'כָּל'
kal_dagesh = 'כָּל'
kal = aleph_bet[52] + vowel[8] + aleph_bet[12]
khal = aleph_bet[10] + vowel[8] + aleph_bet[12]
kol = aleph_bet[52] + vowel[20] + aleph_bet[12]
kol_maqqaf = aleph_bet[52] + vowel[20] + aleph_bet[12] + '־'
khol_maqqaf = aleph_bet[10] + vowel[20] + aleph_bet[12] + '־'
kol_space = aleph_bet[52] + vowel[20] + aleph_bet[12] + ' '
khol_space = aleph_bet[10] + vowel[20] + aleph_bet[12] + ' '

et = 'אֶת'
ve_et = 'וְאֶת'
space_et_space = ' ' + 'אֶת' + ' '
space_ve_et_space = ' ' + 'וְאֶת' + ' '
et_maqqaf = ' ' + 'אֶת' + '־'
ve_et_maqqaf = ' ' + 'וְאֶת' + '־'

In [5]:
shva_exceptions = ['שְׁתֵּי','שְׁתָּיִם','שְׁתַּיִם','שְׁנִַים','שְׁנֵי','שְׁתֵּים','שְׁנֵים','שְׁנֵי','שְׁנֵים','שְׁנָיִם']
battim = 'בָּתִּים'
batteinu = 'בָתֵּֽינוּ'
vattim = 'בָתִּים'
battei = 'בָּתֵּי'
ana = 'אָנָּא'
ana2 = 'אָנָּא'
ana3 = 'אָנָּא'
anah = 'אָנָּה'
kamatz_katan_exceptions = [battim, vattim, ana, ana2, ana3, anah, batteinu]
known_nasog_achor = ['היתה','אזרו']
known_shva_nach = ['לילה']

In [6]:
#removes trop and vowels
def nonalpha_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter.isalpha() == True:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [7]:
#removes trop but not vowels
def trop_remover(word):
    no_cant_word = ''
    for letter in word:
        if letter not in cant:
            no_cant_word = no_cant_word + letter
    return no_cant_word

In [8]:
#converts the 4-letter shem to double-yod, preserving trop and prefixes

def shem_converter(word):
    prefix = ''
    if word[0] != aleph_bet[9]:
        prefix = prefix + word[0]
        i = 1
        if word[1].isalpha() == False:
            prefix = prefix + word[1]
            if word[2].isalpha() == False:
                prefix = prefix + word[2]
    no_prefix_word = word[len(prefix):]

    if len(prefix) > 0:
        yud1 = aleph_bet[9]
    else:
        yud1 = aleph_bet[9]+vowel[0]

    yud2 = aleph_bet[9]+vowel[8]

    cant_marks = []

    #finds suffix
    suffix = ''
    i = -1
    while word[i] != aleph_bet[4]:
        if word[i] in special_characters:
            suffix = word[i] + suffix
        i = i-1
    
    for character in no_prefix_word:
        if character in cant:
            cant_marks.append(character)  
    if len(cant_marks) == 1:
        new_shem = prefix + yud1 + yud2 + cant_marks[0] + suffix
    elif len(cant_marks) == 2:
        new_shem = prefix + yud1 + cant_marks[0] + yud2 + cant_marks[1] + suffix
    else:
        new_shem = prefix + yud1 + yud2 + suffix

    return new_shem

In [9]:
#creates paragraph with new shem using function above
#this needs to be fixed to stop eating special characters (i.e. brackets, \)

def convert_shem(paragraph):
    paragraph = str.replace(paragraph, '־', ' ־ ')
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        if shem in nonalpha_remover(word):
            double_yud = shem_converter(word)
            par_list[index] = double_yud

    new_par = ' '.join(par_list)
    new_par = str.replace(new_par, ' ־ ','־')
    new_par = str.replace(new_par, '׃', '׃')
    return new_par

In [10]:
def na_marker(paragraph,index, mark):
    if paragraph[index-1] == mark or paragraph[index+1] == mark or paragraph[index+2] == mark or paragraph[index-2]==mark:
        return paragraph
        #does nothing, if the shva is already marked with a rafe
    else:
        par_start = paragraph[:index]
        par_end = paragraph[index:]
        new_par = par_start+mark+par_end
        return new_par
        #adds a rafe over the input letter

In [11]:
def long_vowel_checker(word,i,index):
    previous_vowel = word[i]
    while word[i] not in aleph_bet:
        i = i-1
    word_sec = word[i:index]
    #if there's no trop and the previous syllable has a meteg, it's a long vowel with primary stress and the shva is nach
    if any(trop in cant2 for trop in word)==False and meteg in word_sec:
        return False
    trop_count = 0
    for char in word:
        if char in cant2:
            trop_count = trop_count+1
    #if there's 2 trops and 1 is on the previous syllable, the 2nd will be on the last syllable
    #therefore the 1st will be a long vowel with secondary stress, and the shva will be na
    #and this second trope will be on the last syllable, so the shva following a syllable with a trop will be the first one
    #and stress will be secondary, so the shva will be na
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==True:
        return True
    if trop_count == 2 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's one trop on the word and it's not on the previous syllable, it's unstressed, and the shva is na
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==False:
        return True
    #if there's a trop on the previous syllable, assume it's stressed and nach, but it could be nasog achor and na
    #therefore these will be printed for manual checking
    elif trop_count == 1 and any(trop in cant2 for trop in word_sec)==True:
        bare_word = nonalpha_remover(word)
        for element in known_nasog_achor:
            if element in bare_word:
                return True
        for element in known_shva_nach:
            if element in bare_word:
                return True
        print('Check shva in',word,'for nasog achor')
        return False
    elif trop_count == 0 and meteg not in word_sec:
        return True
    else:
        print('Check shva in',word)

In [12]:
#Determines what shvas are na' in a word, and calls the program to mark them
def shva_na_function(word, mark):
    nt_word = trop_remover(word)
    #this skips instances of shem hashem
    if shem_vowels in nt_word or yy_vowels in nt_word:
        return word
    for excep in shva_exceptions:
        if excep in nt_word:
            return word
    if shva not in word:
        return word
    start = 1
    ha_meteg = 'הַֽ'
    shuruk_meteg = 'וּֽ'
    if word[0:3] == ha_meteg or word[0:3] == shuruk_meteg:
        shva_loc = word.find(shva)
        word = na_marker(word,shva_loc, mark)
        start = shva_loc+1
    
    for index in range(start,len(word)-2):
        i = index
        if word[index] != shva:
            continue
            #this doesn't bother with the loop if the letter isn't a shva
        if word[i+1] not in all_char:
            break
        while word[i] not in aleph_bet or word[i]== dagesh:
            i = i-1
        previous_consonant = word[i]
        #if the previous consonant is the beginning of the word, the shva is na'
        if i == 0:
            word = na_marker(word,index, mark)
            continue
        if word[i-1] not in all_char:
            word = na_marker(word,index, mark)
            continue
        
        if word[i-1] == maqqaf:
            word = na_marker(word,index, mark)
        #if the previous consonant has a dagesh, the shva must be na'
        if previous_consonant in letter_with_dagesh:
            word = na_marker(word,index, mark)
            continue
        if word[i+1] == dagesh or word[index+1] == dagesh:
            word = na_marker(word,index, mark)
            continue
            
        #determines the next consonant
        i = index + 1
        while word[i] not in aleph_bet and i<len(word)-1:
            i = i+1
        next_consonant = word[i]
        if next_consonant == previous_consonant:
            word = na_marker(word,index, mark)
            continue
        i=0
        while i<5:
            if word[index]==letters_w_final[i] and next_consonant == final_letter[i]:
                word = na_marker(word,index, mark)
                continue   
            i=i+1

        #determines the previous vowel
        i = index-2
        while word[i] not in vowels_limited and i>0:
            i = i-1
        if i == 0:
            continue
        else:
            previous_vowel = word[i]
            #if the previous vowel is a shva, the current shva is na'
            if previous_vowel == shva:
                word = na_marker(word,index, mark)
                continue
            if previous_vowel in long_vowels and long_vowel_checker(word,i,index)==True:
                word = na_marker(word,index, mark)   
        
        #a shva following a long vowel is na', unless the long vowel is word-initial shuruk
   

    #figure out plan for סקינמלוי letters after הַ־ without dagesh
    #figure out plan for letters following shuruk with meteg
    #for now, neither of these get a shva na
    return word

In [13]:
#Calls the shva_na_function for each word in the input paragraph
def shva_na_converter(paragraph, mark):
    par_list = paragraph.split()
    for word in par_list:
        index = par_list.index(word)
        par_list[index] = shva_na_function(word, mark)
    new_par = ' '.join(par_list)
    return new_par

In [14]:
shortnames = {
    "מלכים א":"מ״א",
    "מלכים ב":"מ״ב",
    "שמואל א":"ש״א",
    "שמואל א":"ש״ב",
    "בראשית":"ברא׳",
    "ישעיהו":"ישעיה",
    "ירמיהו":"ירמיה",
    "יחזקאל":"יחזקﭏ",
    "שיר השירים":"שה״ש",
    "דברי הימים א":"דה״א",
    "דברי הימים ב":"דה״ב"
}

def shortentitle(title):
    for entry in shortnames:
        if entry in title:
            title = title.replace(entry,shortnames[entry])
    if title[-1] == "׳":
        title = title[0:-1]
    if title[-2] == "״":
        title = title[0:-2]+title[-1]
    return title

In [28]:
def textpuller(text,version):
    link = "https://www.sefaria.org/api/texts/"+text+"?vhe="+version+"&context=0"
    print(link)
    with urllib.request.urlopen(link) as url:
        sefariaresult = json.loads(url.read().decode())
    text = ""
    if type(sefariaresult["he"])==str:
        text = sefariaresult["he"]+" "
    elif type(sefariaresult["he"])==list:
        for verse in sefariaresult["he"]:
            text = text + verse + " "
    title_he = sefariaresult["heSectionRef"]
    return text,title_he

#figure out how to deal with formatting

In [29]:
def removeformatting(text):
    while "<" in text and ">" in text:
        loc1 = text.find("<")
        loc2 = text.find(">",loc1)+1
        text = text.replace(text[loc1:loc2],"")
    return text

In [30]:
heversion = "Miqra_according_to_the_Masorah"

In [31]:
def returnparam(maintext,string):
    if string not in maintext:
        return 
    loc1 = maintext.find("&"+string+"=")
    start = maintext.find("=",loc1)+1
    end = maintext.find("&",loc1+1)
    return maintext[start:end]

In [45]:
def TeXcommand(commandname,textname):
    sefariaresult = textpuller(textname,heversion)
    verses = removeformatting(sefariaresult[0])
    title = shortentitle(sefariaresult[1])
    
    #this converts shem
    #you'd add the shva na marking function here if you wanted
    verses = convert_shem(verses)
    end = verses.rfind("׃")+1
    verses = verses[0:end+1]
    outputstring = "\\newcommand{\\"+commandname+"}{%\n"
    sourcestring = "\source{"+title+"}"
    first_space = verses.find(" ")
    outputstring += verses[0:first_space]+sourcestring+verses[first_space:-1]
    outputstring += "}"
    return outputstring

def TeXtext(textname):
    sefariaresult = textpuller(textname,heversion)
    verses = removeformatting(sefariaresult[0])
    title = shortentitle(sefariaresult[1])
    
        #this converts shem
    #you'd add the shva na marking function here if you wanted
    verses = convert_shem(verses)
    end = verses.rfind("׃")+1
    verses = verses[0:end]
    sourcestring = "\source{"+title+"}"
    first_space = verses.find(" ")
    outputstring = verses[0:first_space]+sourcestring+verses[first_space:]+'\n'
    return outputstring

In [46]:
def TeXparser(inputtext):
    if inputtext[0:3] != "%?&":
        return None,None
    if "name" in inputtext:
        commandname = returnparam(inputtext,"name")
    else:
        commandname = None
    if "text" in inputtext:
        textname = returnparam(inputtext,"text")
    else:
        textname = None
    return commandname,textname

In [47]:
##using the above to generate a commands

In [48]:
def TeXconverter(inputfile,outputfile):
    with open(inputfile, 'r', encoding='utf-8') as infile:
        lines = list(infile.readlines())
    output_lines = []
    for line in lines:
        parsed_line = TeXparser(line)
        if parsed_line[1] == None:
            output_lines.append(line)
        elif parsed_line[0] == None:
            output_lines.append(TeXtext(parsed_line[1]))
        else:
            output_lines.append(TeXcommand(parsed_line[0],parsed_line[1]))
    with open(outputfile, 'w', encoding='utf-8') as outfile:
        for line in output_lines:
            outfile.write(line)

In [49]:
#TeXconverter('to_include/commands_raw.tex','to_include/commands.tex')

In [50]:
#TeXconverter('chapters/PDZ_raw.tex','chapters/PDZ_test.tex')

https://www.sefaria.org/api/texts/Psalms.99:5?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.99:9?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.78:38?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.40:12?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.25:6?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.68:35-36?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.94:1-2?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.3:9?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.46:8?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.84:13?vhe=Miqra_according_to_the_Masorah&context=0
https://www.sefaria.org/api/texts/Psalms.20:10?vhe=Miqra_according_to_the_Masorah&

In [52]:
verses = []
verses.append("Psalms.106:47")
verses.append("Psalms.130:3-4")
verses.append("Jeremiah.14:7")
verses.append("Psalms.25:6")
verses.append("Psalms.20:10")
verses.append("Psalms.106:47")
verses.append("Daniel.9:15-17")
verses.append("Daniel.9:18-19")
verses.append("Isaiah.64:7")
verses.append("Joel.2:17")
verses.append("Jeremiah.14:9")
verses.append("Exodus.32:12")
verses.append("Daniel.9:7")
verses.append("Psalms.118:25")

for verse in verses:
    print(TeXtext(verse))

https://www.sefaria.org/api/texts/Psalms.106:47?vhe=Miqra_according_to_the_Masorah&context=0
הוֹשִׁיעֵ֨נוּ\source{תהילים קו} ׀ יְ֘יָ֤ אֱלֹהֵ֗ינוּ וְקַבְּצֵנוּ֮ מִֽן־הַגּ֫וֹיִ֥ם לְ֭הֹדוֹת לְשֵׁ֣ם קׇדְשֶׁ֑ךָ לְ֝הִשְׁתַּבֵּ֗חַ בִּתְהִלָּתֶֽךָ׃

https://www.sefaria.org/api/texts/Psalms.130:3-4?vhe=Miqra_according_to_the_Masorah&context=0
אִם־עֲוֺנ֥וֹת\source{תהילים קל} תִּשְׁמׇר־יָ֑הּ אֲ֝דֹנָ֗י מִ֣י יַעֲמֹֽד׃ כִּֽי־עִמְּךָ֥ הַסְּלִיחָ֑ה לְ֝מַ֗עַן תִּוָּרֵֽא׃

https://www.sefaria.org/api/texts/Jeremiah.14:7?vhe=Miqra_according_to_the_Masorah&context=0
אִם־עֲוֺנֵ֙ינוּ֙\source{ירמיה יד} עָ֣נוּ בָ֔נוּ יְיָ֕ עֲשֵׂ֖ה לְמַ֣עַן שְׁמֶ֑ךָ כִּי־רַבּ֥וּ מְשׁוּבֹתֵ֖ינוּ לְךָ֥ חָטָֽאנוּ׃

https://www.sefaria.org/api/texts/Psalms.25:6?vhe=Miqra_according_to_the_Masorah&context=0
זְכֹר־רַחֲמֶ֣יךָ\source{תהילים כה} יְיָ֭ וַחֲסָדֶ֑יךָ כִּ֖י מֵעוֹלָ֣ם הֵֽמָּה׃

https://www.sefaria.org/api/texts/Psalms.20:10?vhe=Miqra_according_to_the_Masorah&context=0
יְיָ֥\source{תהילים כ} הוֹשִׁ֑יעָה הַ֝מֶּ֗לֶךְ יַעֲנֵ֥נוּ